# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [13]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [14]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [15]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [16]:
# Calculate the mean values for each neighborhood and concat with coordinates for the Neighbourhood Map
avg_house_values = to_data[["average_house_value", "neighbourhood"]].groupby("neighbourhood").mean().reset_index()
avg_house_values

avg_house_value_locations = pd.concat([avg_house_values, df_neighbourhood_locations], axis="columns", sort=True)
avg_house_value_locations.columns = ["drop", "average_house_value", "neighbourhood", "lat", "lon"]
avg_house_value_locations.drop(columns=["drop"], inplace=True)

# Calculate the sum number of dwelling types by year
sum_dwellings = to_data.groupby("year").sum()
sum_dwellings_by_year = sum_dwellings.drop(columns=["average_house_value", "shelter_costs_owned", "shelter_costs_rented"])
sum_dwellings_by_year

# Calculate the average monthly shelter costs for owned and rented dwellings
avg_owned_cost = to_data[["shelter_costs_owned"]].groupby("year").mean()
avg_rented_cost = to_data[["shelter_costs_rented"]].groupby("year").mean()

# Calculate the average house value per year
avg_house_value_by_year = to_data[['average_house_value']].groupby(["year"]).mean()
avg_house_value_by_year

# Calculate the average house value per year by neighbourhood
by_neighbourhood = to_data[["average_house_value", "neighbourhood"]].groupby(["neighbourhood", "year"]).mean()
by_neighbourhood = by_neighbourhood.reset_index()

# Calculate the number of dwelling types per year
num_dwelling_types = to_data.drop(columns=["average_house_value", "shelter_costs_owned", "shelter_costs_rented"]).groupby(["neighbourhood", "year"]).mean()
num_dwelling_types.head()

# Calculate the mean number of dwelling types units per year
avg_dwelling_types_by_year = to_data.groupby("year").mean()
avg_dwelling_types_by_year

# Getting the data from the top 10 expensive neighbourhoods
avg_house_values = to_data[["average_house_value", "neighbourhood"]].groupby(["neighbourhood"]).mean()
avg_house_values.sort_values(by="average_house_value", inplace=True, ascending=False)
top_10_most_expensive = avg_house_values.iloc[:10]

# Prepping dataframe for sunburst chart

## Get top 10 most expensive subrubs for each year
# 2001
df = to_data[["average_house_value", "neighbourhood"]]
top_10_expensive_2001 = df.loc[2001].sort_values(by="average_house_value", ascending=False).iloc[:10]

# 2006
top_10_expensive_2006 = df.loc[2006].sort_values(by="average_house_value", ascending=False).iloc[:10]

# 2011
top_10_expensive_2011 = df.loc[2011].sort_values(by="average_house_value", ascending=False).iloc[:10]

# 2016
top_10_expensive_2016 = df.loc[2016].sort_values(by="average_house_value", ascending=False).iloc[:10]

sunburst_data = pd.concat([top_10_expensive_2001, top_10_expensive_2006, top_10_expensive_2011, top_10_expensive_2016]).reset_index()

### Panel Visualization Functions

In [17]:
# Define Panel visualization functions
def neighbourhood_map():
    """Neighbourhood Map"""
    
    avg_house_value_map = px.scatter_mapbox(
            avg_house_value_locations,
            lat="lat",
            lon="lon",
            color="average_house_value",
            size="average_house_value",
            zoom=9,
            height=700,
            width=1000,
            text="neighbourhood",
            title="Average House Values in Toronto",
            color_continuous_scale="darkmint")
    return avg_house_value_map

def create_bar_chart(data, title, xlabel, ylabel, color):
    """
    Create a barplot based in the data argument.
    """
    
    bar_chart = data.hvplot.bar(title=title, 
                                color=color, 
                                rot=45, 
                                xlabel=xlabel, 
                                ylabel=ylabel)
    bar_chart.opts(height=375, width=480, yformatter="%.0f")
    return bar_chart
    

def create_line_chart(data, title, xlabel, ylabel, color):
    """
    Create a line chart based in the data argument.
    """
    
    line_chart = data.hvplot(title=title, 
                             xlabel=xlabel, 
                             ylabel=ylabel, 
                             color=color, 
                             yformatter="%.0f",)
    line_chart.opts(height=375, width=520, fontsize={"title":13, "ylabel":10, "xlabel":10})
    return line_chart

def average_house_value():
    """Average house values per year."""
    
    line_chart = create_line_chart(avg_house_value_by_year, "Average House Value in Toronto", "Average House Value", "Year", "cyan").opts(ylim=(0,800000), fontsize={"title":11})
    return line_chart
    ## ylim for this chart is bugged and not working, chart renders first with 1.5m ylim and then if you hit refresh, ylim paramater kicks in

def average_value_by_neighbourhood():
    """Average house values by neighbourhood."""
    
    chart = by_neighbourhood.hvplot(x="year", y="average_house_value", 
                                    groupby="neighbourhood", 
                                    yformatter="%.0f").opts(
            height=375, width=520, fontsize={"title":13, "ylabel":10, "xlabel":10}, xlabel="Year", ylabel="Average House Value", color="blue")
    return chart

def number_dwelling_types():
    """Number of dwelling types per year"""
    
    selected_columns = ["single_detached_house", "apartment_five_storeys_plus", "movable_dwelling", "semi_detached_house", "row_house", "duplex", "apartment_five_storeys_less", "other_house"]
    
    chart = num_dwelling_types.hvplot.bar(x="year", 
                                          y=selected_columns, 
                                          rot=90, 
                                          groupby="neighbourhood").opts(
        height=375, width=520, fontsize={"title":13, "ylabel":10, "xlabel":10}, xlabel="Year/Type of Dwellings", ylabel="Dwelling Type Units")
    return chart
    
def average_house_value_snapshot():
    """Average house value for all Toronto's neighbourhoods per year."""
    
    chart = px.bar(
            by_neighbourhood,
            x="neighbourhood",
            y="average_house_value",
            facet_row="year",
            color="average_house_value",
            labels={"neighbourhood":"Neighbourhood", "average_house_value":"Average Value"},
            color_continuous_scale="darkmint",
            height=750,
            width=520
        )
    return chart

def top_most_expensive_neighbourhoods():
    """Top 10 most expensive neighbourhoods."""
    
    chart = create_bar_chart(top_10_most_expensive, "Top 10 Most Expensive Neighbourhoods in Toronto", "Average House Value", "Neighbourhood", "blue").opts(ylim=(0,1600000), height=450, width=650)
    return chart

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [18]:
# Create a Title for the Dashboard
dashboard_title = "Real Estate Analysis of Toronto from 2001 to 2016"

# Define a welcome text
welcome_text = """This dashboard presents a visual analysis of historical housevalues, dwelling types per neighbourhood and dwelling costs in Toronto, Ontario
                  according to census data from 2001 to 2016. You can navigate through the tabs above to explore more details about the evolution of the real estate
                  market across these 6 years.
               """

# Create a tab layout for the dashboard

# Map pane
map_pane = pn.pane.Plotly(neighbourhood_map())
map_column = pn.Column(welcome_text, map_pane)

# Dwelling type rows and column
dwelling_type_row_1 = pn.Row(create_bar_chart(sum_dwellings_by_year.loc[2001], "Dwelling Types in Toronto in 2001", "Dwelling Types", "Dwelling Type Units", "red").opts(ylim=(0, 600000)),
                             create_bar_chart(sum_dwellings_by_year.loc[2006], "Dwelling Types in Toronto in 2006", "Dwelling Types", "Dwelling Type Units", "blue").opts(ylim=(0, 600000)))
dwelling_type_row_2 = pn.Row(create_bar_chart(sum_dwellings_by_year.loc[2011], "Dwelling Types in Toronto in 2011", "Dwelling Types", "Dwelling Type Units", "green").opts(ylim=(0, 600000)),
                             create_bar_chart(sum_dwellings_by_year.loc[2016], "Dwelling Types in Toronto in 2016", "Dwelling Types", "Dwelling Type Units", "cyan").opts(ylim=(0, 600000)))
dwelling_type_column = pn.Column(dwelling_type_row_1, dwelling_type_row_2)

# Average values column
average_value_row = pn.Row(create_line_chart(avg_owned_cost, "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue").opts(fontsize={"title":11}),
                           create_line_chart(avg_rented_cost, "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "green").opts(fontsize={"title":11}))
average_value_column = pn.Column(average_value_row, average_house_value())

# Neighbourhood rows and columns
by_neighbourhood_column = pn.Column(average_value_by_neighbourhood(), number_dwelling_types())
by_neighbourhood_row = pn.Row(by_neighbourhood_column, average_house_value_snapshot())

# Expensive neighbourhood row
expensive_neighbourhood_row = pn.Row(top_most_expensive_neighbourhoods()) 


# Create the main dashboard
dashboard = pn.Tabs(
    ("Welcome", map_column),
    ("Yearly Market Analysis", dwelling_type_column),
    ("Shelter Costs Vs. House Values", average_value_column),
    ("Neighbourhood Analysis", by_neighbourhood_row),
    ("Top Expensive Neighbourhoods", expensive_neighbourhood_row)
)

## Serve the Panel Dashboard

In [19]:
dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [1] Column
        [0] Row
            [0] HoloViews(Bars)
            [1] HoloViews(Bars)
        [1] Row
            [0] HoloViews(Bars)
            [1] HoloViews(Bars)
    [2] Column
        [0] Row
            [0] HoloViews(Curve)
            [1] HoloViews(Curve)
        [1] HoloViews(Curve)
    [3] Row
        [0] Column
            [0] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                    [1] VSpacer()
            [1] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                    [1] VSpacer()
        [1] Plotly(Figure)
    [4] Row
        [0] HoloViews(Bars)

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# neighbourhood_map().show()

In [ ]:
# create_bar_chart(data, title, xlabel, ylabel, color)

# # Bar chart for 2001
# create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
# create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [ ]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [ ]:
# average_house_value()

In [ ]:
# average_value_by_neighbourhood()

In [ ]:
# number_dwelling_types()

In [ ]:
# average_house_value_snapshot()

In [ ]:
# top_most_expensive_neighbourhoods()

In [ ]:
# sunburts_cost_analysis()